In [183]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import hvplot.pandas
import datetime as dt 
import holoviews as hv


In [184]:
import warnings
warnings.simplefilter(action='ignore', category=(UserWarning, FutureWarning))


In [185]:
#get realestate data
tsla = yf.Ticker('tsla').history(period='2mo', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)

In [186]:
tsla.head()

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-23 00:00:00-05:00,135.869995,145.380005,134.270004,143.750000,203119200
2023-01-24 00:00:00-05:00,143.000000,146.500000,141.100006,143.889999,158699100
2023-01-25 00:00:00-05:00,141.910004,146.410004,138.070007,144.429993,192734300
2023-01-26 00:00:00-05:00,159.970001,161.419998,154.759995,160.270004,234815100
2023-01-27 00:00:00-05:00,162.429993,180.679993,161.169998,177.899994,306590600


In [187]:
#tsla["Date"] = pd.to_datetime(tsla['Date']).dt.date
tsla.index = pd.to_datetime(tsla.index).date
tsla.head(20)

,Open,High,Low,Close,Volume
2023-01-23,135.869995,145.380005,134.270004,143.750000,203119200
2023-01-24,143.000000,146.500000,141.100006,143.889999,158699100
2023-01-25,141.910004,146.410004,138.070007,144.429993,192734300
2023-01-26,159.970001,161.419998,154.759995,160.270004,234815100
2023-01-27,162.429993,180.679993,161.169998,177.899994,306590600
2023-01-30,178.050003,179.770004,166.500000,166.660004,230878800
2023-01-31,164.570007,174.300003,162.779999,173.220001,196813500
2023-02-01,173.889999,183.809998,169.929993,181.410004,213806300
2023-02-02,187.330002,196.750000,182.610001,188.270004,217448300
2023-02-03,183.949997,199.000000,183.690002,189.979996,232662000


In [188]:
current_price = price_history['Close'][-1]
current_price

183.24000549316406

In [189]:
# if the previous price is lower than current price then set values
#up = close_now - close_previous
#down = 0
#down = close_previous - close_now
#use SMA of EMA
# relative strength is calculated by 
# RS = EMA(up)/EMA(down) (Exponential moving avg)
#the end with relative strength index
#RSI = 100-(100/(1-RS))

In [190]:
tsla['delta']= tsla["Close"].diff()
tsla["up"]= tsla['delta'].clip(lower = 0)
tsla['down'] = -1*tsla['delta'].clip(upper = 0)
tsla.head(10)

,Open,High,Low,Close,Volume,delta,up,down
2023-01-23,135.869995,145.380005,134.270004,143.750000,203119200,NaN,NaN,NaN
2023-01-24,143.000000,146.500000,141.100006,143.889999,158699100,0.139999,0.139999,-0.00000
2023-01-25,141.910004,146.410004,138.070007,144.429993,192734300,0.539993,0.539993,-0.00000
2023-01-26,159.970001,161.419998,154.759995,160.270004,234815100,15.840012,15.840012,-0.00000
2023-01-27,162.429993,180.679993,161.169998,177.899994,306590600,17.629990,17.629990,-0.00000
2023-01-30,178.050003,179.770004,166.500000,166.660004,230878800,-11.239990,0.000000,11.23999
2023-01-31,164.570007,174.300003,162.779999,173.220001,196813500,6.559998,6.559998,-0.00000
2023-02-01,173.889999,183.809998,169.929993,181.410004,213806300,8.190002,8.190002,-0.00000
2023-02-02,187.330002,196.750000,182.610001,188.270004,217448300,6.860001,6.860001,-0.00000
2023-02-03,183.949997,199.000000,183.690002,189.979996,232662000,1.709991,1.709991,-0.00000


In [204]:
ema_up = tsla['up'].ewm(com = 1, adjust = False).mean()
ema_down = tsla["down"].ewm(com=1, adjust = False).mean()
ema_up.hvplot() * ema_down.hvplot()

:Overlay
   .Curve.Up   :Curve   [index]   (up)
   .Curve.Down :Curve   [index]   (down)

In [206]:
rs = ema_up/ema_down
tsla['RSI'] = 100 - (100/(1+rs))

tsla['RSI'].hvplot(width = 800)* hv.HLine(70).opts(
    color="red", line_width=2.0
) * hv.HLine(30).opts(
    color="green", line_width=2.0)

:Overlay
   .Curve.RSI :Curve   [index]   (RSI)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]

In [ ]:
If RSI is 